# Proyecto  Final -  Parte 3

### **Parte 3.  Spark Streaming**

- 1. [Importación de librerias](#1)
- 2. [Pasos previos](#2)
- 3. [Análisis electricidad](#3)

En esta parte se van a analizar los datos en tiempo real a través de Spark Streaming, los datos son emitidos desde un servidor ofrecido para llevar acabo este proyecto, para llevar a cabo esta parte solamente se van a analizar datos del puerto número 45001 que corresponden con electricidad, para evitar la redundancia 

<a id="1"></a>
### * Se importan las librerias que se van a utilizar*

In [1]:
import org.apache.spark._

In [2]:
import org.apache.spark.streaming._

<a id="2"></a>
### * Pasos previos al análisis de los datos* 
#### *Se crea el objeto StreamingContext indicando que el intervalo entre cada batch es de 30 segundos y se establece el directorio de checkpoint para Apache Spark *.

In [3]:
val streamctx = new StreamingContext(sc, Seconds(30)) 

streamctx = org.apache.spark.streaming.StreamingContext@263a34bb


org.apache.spark.streaming.StreamingContext@263a34bb

In [4]:
streamctx.checkpoint("/eoi/streaming/checkpoint") 

#### *Se registra la fuente de datos con el correspondiente emisor , en este caso para el puerto indicado va a ser de electricidad*

In [5]:
val elec = streamctx.socketTextStream("emisor.eoi.rbs-net.com", 45001) 

elec = org.apache.spark.streaming.dstream.SocketInputDStream@11f642fb


org.apache.spark.streaming.dstream.SocketInputDStream@11f642fb

#### *Se lleva a cabo una impresion para comprobar que los datos son recibidos de manera correcta*

#### * Se crea un objeto de la clase Electricidad, con un subconjunto de los campos correspondientes a los datos recibidos, porteriormente se realiza un split con ; ya que son emitidos así   y se crea una lista con los valores correspondientes*

In [6]:
case class Electricidad(Tipo: String, Consumo: Double, Año: Int, 
                    Mes: Int, Día: Int, Hora: Int, Minutos: Int)

defined class Electricidad


In [7]:
val dat_elec = elec.flatMap(x => {
    val campos = x.split(";")  
    try {
        List(Electricidad(campos(0),campos(1).toDouble,campos(2).toInt,campos(3).toInt, campos(4).toInt, campos(5).toInt,campos(6).toInt))
    } catch {
        case e : Throwable => {
            // Llamada al sistema de notificación en caso de fallo en la entrada.
        }
        List()
    }
})

dat_elec = org.apache.spark.streaming.dstream.FlatMappedDStream@2b1e1648


org.apache.spark.streaming.dstream.FlatMappedDStream@2b1e1648

<a id="3"></a>
### * Análisis electricidad *

Se seleccionan los campos que me interesa analizar, 0 y 1 consumo por edificio y se agrupan por la clave, que es edificio

In [8]:
val cons_elec = dat_elec.map(x => (x.Tipo, x.Consumo)).reduceByKey(_ + _)

cons_elec = org.apache.spark.streaming.dstream.ShuffledDStream@72805b06


org.apache.spark.streaming.dstream.ShuffledDStream@72805b06

Se crea  la variable resultado_cons_elec que acumulara los consumos por edificio utilizando el método updateStateByKey, el cual permite analizar los datos de entrada teniendo en cuenta el resultado de los anteriores datos de entrada


In [9]:
val resultado_cons_elec = cons_elec.updateStateByKey((entrada, estado:Option[Double]) => {
                                                   if (estado.isEmpty == true) {
                                                       Some(entrada.sum)
                                                   } else {
                                                       Some(entrada.sum + estado.getOrElse(0.0))
                                                   }})

org.apache.spark.streaming.dstream.StateDStream@437c64c1

resultado_cons_elec = org.apache.spark.streaming.dstream.StateDStream@437c64c1


In [10]:
resultado_cons_elec.print()

In [21]:
// streamctx.start()

-------------------------------------------
Time: 1580297940000 ms
-------------------------------------------
(ELEC_EDIF_17_COMEDOR,40.79999999999999)
(ELEC_EDIF_18_KIOSCO,12.430000000000001)

-------------------------------------------
Time: 1580297970000 ms
-------------------------------------------
(ELEC_EDIF_17_COMEDOR,40.79999999999999)
(ELEC_EDIF_18_KIOSCO,134.18699999999993)

-------------------------------------------
Time: 1580298000000 ms
-------------------------------------------
(ELEC_EDIF_17_COMEDOR,40.79999999999999)
(ELEC_EDIF_18_KIOSCO,143.20899999999992)
(ELEC_EDIF_2,353.839)

-------------------------------------------
Time: 1580298030000 ms
-------------------------------------------
(ELEC_EDIF_17_COMEDOR,40.79999999999999)
(ELEC_EDIF_18_KIOSCO,143.20899999999992)
(ELEC_EDIF_20,267.222)
(ELEC_EDIF_2,842.952)

-------------------------------------------
Time: 1580298060000 ms
-------------------------------------------
(ELEC_EDIF_17_COMEDOR,40.79999999999999)
(ELEC

### * Se muestran los edificios que más consumos han transmitido*

In [11]:
val resultado_max = resultado_cons_elec.transform(x => x.sortBy(x => x._2, false))

resultado_max = org.apache.spark.streaming.dstream.TransformedDStream@242f6065


org.apache.spark.streaming.dstream.TransformedDStream@242f6065

In [12]:
resultado_max.print()

In [24]:
//streamctx.start()

-------------------------------------------
Time: 1580298780000 ms
-------------------------------------------
(ELEC_UPO,2978.32)

-------------------------------------------
Time: 1580298810000 ms
-------------------------------------------
(ELEC_UPO,16352.699000000002)

-------------------------------------------
Time: 1580298840000 ms
-------------------------------------------
(ELEC_UPO,16352.699000000002)

-------------------------------------------
Time: 1580298870000 ms
-------------------------------------------
(ELEC_UPO,16352.699000000002)

-------------------------------------------
Time: 1580298900000 ms
-------------------------------------------
(ELEC_UPO,16352.699000000002)

-------------------------------------------
Time: 1580298930000 ms
-------------------------------------------
(ELEC_UPO,16352.699000000002)

-------------------------------------------
Time: 1580298960000 ms
-------------------------------------------
(ELEC_UPO,16352.699000000002)

-----------------

### * Se muestra el número de medidas producidas en el ultimo mínumo con desplazamiento de 30 segundos*

In [13]:
val min_3_consum = cons_elec.reduceByKeyAndWindow((a:Double, b:Double) => a + b, Seconds(60), Seconds(30))

min_3_consum = org.apache.spark.streaming.dstream.ShuffledDStream@575f24e0


org.apache.spark.streaming.dstream.ShuffledDStream@575f24e0

In [14]:
min_3_consum.print()

In [18]:
// streamctx.start()

-------------------------------------------
Time: 1580302410000 ms
-------------------------------------------

-------------------------------------------
Time: 1580302410000 ms
-------------------------------------------

-------------------------------------------
Time: 1580302410000 ms
-------------------------------------------

-------------------------------------------
Time: 1580302410000 ms
-------------------------------------------

-------------------------------------------
Time: 1580302440000 ms
-------------------------------------------

-------------------------------------------
Time: 1580302440000 ms
-------------------------------------------

-------------------------------------------
Time: 1580302440000 ms
-------------------------------------------

-------------------------------------------
Time: 1580302440000 ms
-------------------------------------------

-------------------------------------------
Time: 1580302470000 ms
-------------------------------------

-------------------------------------------
Time: 1580302860000 ms
-------------------------------------------
Electricidad(ELEC_EDIF_1,8.322,2013,10,15,15,0)
Electricidad(ELEC_EDIF_1,7.272,2013,10,15,15,15)
Electricidad(ELEC_EDIF_1,7.229,2013,10,15,15,30)
Electricidad(ELEC_EDIF_1,6.875,2013,10,15,15,45)
Electricidad(ELEC_EDIF_1,7.206,2013,10,15,16,0)
Electricidad(ELEC_EDIF_1,7.389,2013,10,15,16,15)
Electricidad(ELEC_EDIF_1,6.912,2013,10,15,16,30)
Electricidad(ELEC_EDIF_1,7.417,2013,10,15,16,45)
Electricidad(ELEC_EDIF_1,7.428,2013,10,15,17,0)
Electricidad(ELEC_EDIF_1,7.683,2013,10,15,17,15)
Electricidad(ELEC_EDIF_1,6.712,2013,10,15,17,30)
Electricidad(ELEC_EDIF_1,8.524,2013,10,15,17,45)
Electricidad(ELEC_EDIF_1,8.584,2013,10,15,18,0)
Electricidad(ELEC_EDIF_1,8.765,2013,10,15,18,15)
Electricidad(ELEC_EDIF_1,8.465,2013,10,15,18,30)
Electricidad(ELEC_EDIF_1,8.621,2013,10,15,18,45)
Electricidad(ELEC_EDIF_1,4.902,2013,10,15,19,0)
Electricidad(ELEC_EDIF_1,4.51,2013,10,15,19,15)
Electricidad(

### *Análisis en Streaming*

En cada batch se mostrará
- Acumulación los consumos por edificio
- Los edificios que más consumos han transmitido
- Número de medidas producidas en el ultimo mínumo con desplazamiento de 30 segundos

In [15]:
streamctx.start()

-------------------------------------------
Time: 1580303130000 ms
-------------------------------------------
(ELEC_EDIF_14,564.233)

-------------------------------------------
Time: 1580303130000 ms
-------------------------------------------
(ELEC_EDIF_14,564.233)

-------------------------------------------
Time: 1580303130000 ms
-------------------------------------------
(ELEC_EDIF_14,564.233)

-------------------------------------------
Time: 1580303160000 ms
-------------------------------------------
(ELEC_EDIF_15,303.429)
(ELEC_EDIF_14,582.5749999999999)

-------------------------------------------
Time: 1580303160000 ms
-------------------------------------------
(ELEC_EDIF_14,582.5749999999999)
(ELEC_EDIF_15,303.429)

-------------------------------------------
Time: 1580303160000 ms
-------------------------------------------
(ELEC_EDIF_15,303.429)
(ELEC_EDIF_14,582.5749999999999)

-------------------------------------------
Time: 1580303190000 ms
------------------------

Para parar el proceso

In [16]:
streamctx.stop()

Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor$$anonfun$restartReceiver$1.apply$mcV$sp(ReceiverSupervisor.scala:196)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor$$anonfun$restartReceiver$1.apply(ReceiverSupervisor.scala:189)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor$$anonfun$restartReceiver$1.apply(ReceiverSupervisor.scala:189)
	at scala.concurrent.impl.Future$PromiseCompletingRunnable.liftedTree1$1(Future.scala:24)
	at scala.concurrent.impl.Future$PromiseCompletingRunnable.run(Future.scala:24)
	at java.util.concurrent.T